# 1、数据加载和预处理

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 加载数据
data = pd.read_csv('./data/data.csv')

# 查看数据基本信息
print(data.head())
print(data.shape)

   id diagnosis_result  radius  texture  perimeter  area  smoothness  \
0   1                M      23       12        151   954       0.143   
1   2                B       9       13        133  1326       0.143   
2   3                M      21       27        130  1203       0.125   
3   4                M      14       16         78   386       0.070   
4   5                M       9       19        135  1297       0.141   

   compactness  symmetry  fractal_dimension  
0        0.278     0.242              0.079  
1        0.079     0.181              0.057  
2        0.160     0.207              0.060  
3        0.284     0.260              0.097  
4        0.133     0.181              0.059  
(100, 10)


In [13]:
# 处理特征和标签
X = data.iloc[:, 2:].values  # 提取特征（从radius列开始）
y = data['diagnosis_result'].values  # 提取标签（M为恶性，B为良性）

# 将标签转换为数字（M=1, B=0）
y = np.array([1 if label == 'M' else 0 for label in y])

# 分割训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 特征标准化,消除特征尺度差异的影响
scaler = StandardScaler()
print("Before scaling: ", X_train[0])
X_train = scaler.fit_transform(X_train)
print("After scaling: ", X_train[0])
X_test = scaler.transform(X_test)

Before scaling:  [1.80e+01 1.30e+01 7.30e+01 4.09e+02 9.50e-02 5.50e-02 1.92e-01 5.90e-02]
After scaling:  [ 0.21131834 -0.96673649 -0.89158244 -0.80594761 -0.46541627 -1.08266423
 -0.02309829 -0.66942275]


# 2、KNN算法实现

In [14]:
class KNN:
    """
    K-近邻（K-Nearest Neighbors）分类器。

    参数:
        k (int, 可选): 最近邻居的数量。默认为 5。
        distance_metric (str, 可选): 用于计算样本之间距离的度量方式。
                                    支持 'euclidean' (欧氏距离) 和 'manhattan' (曼哈顿距离)。
                                    默认为 'euclidean'。
    """
    def __init__(self, k=5, distance_metric='euclidean'):
        """
        初始化 KNN 分类器。
        """
        self.k = k
        self.distance_metric = distance_metric
        self.X_train = None  # 存储训练特征数据
        self.y_train = None  # 存储训练标签数据

    def fit(self, X_train, y_train):
        """
        存储训练数据。

        参数:
            X_train (numpy.ndarray): 训练特征数据，形状为 (n_samples, n_features)。
            y_train (numpy.ndarray): 训练标签数据，形状为 (n_samples,)。

        返回:
            self: 返回 KNN 实例自身。
        """
        self.X_train = X_train
        self.y_train = y_train
        return self

    def calculate_distance(self, x1, x2):
        """
        计算两个样本之间的距离。

        参数:
            x1 (numpy.ndarray): 第一个样本的特征向量，形状为 (n_features,)。
            x2 (numpy.ndarray): 第二个样本的特征向量，形状为 (n_features,)。

        返回:
            float: 两个样本之间的距离。

        Raises:
            ValueError: 如果指定的距离度量方式不支持。
        """
        if self.distance_metric == 'euclidean':
            # 欧氏距离公式：sqrt(sum((a_i - b_i)^2))
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.distance_metric == 'manhattan':
            # 曼哈顿距离公式：sum(|a_i - b_i|)
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("不支持的距离度量方式")

    def predict(self, X):
        """
        预测新样本的类别。

        参数:
            X (numpy.ndarray): 要预测的样本特征数据，形状为 (n_samples, n_features)。

        返回:
            numpy.ndarray: 包含每个样本预测类别的数组，形状为 (n_samples,)。
        """
        # 对每个输入样本进行预测
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        """
        预测单个样本的类别。

        参数:
            x (numpy.ndarray): 单个要预测的样本特征向量，形状为 (n_features,)。

        返回:
            int or str: 预测的类别标签。
        """
        # 1. 计算输入样本与所有训练样本的距离
        distances = [self.calculate_distance(x, x_train) for x_train in self.X_train]

        # 2. 获取距离最近的 k 个邻居的索引
        # np.argsort(distances) 返回 distances 数组排序后的索引
        # [:self.k] 取前 k 个最小距离的索引
        k_indices = np.argsort(distances)[:self.k]

        # 3. 获取这 k 个邻居的标签
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # 4. 返回出现次数最多的标签（投票）
        # np.unique 返回数组中唯一的元素以及每个元素的计数
        unique_labels, counts = np.unique(k_nearest_labels, return_counts=True)
        # np.argmax(counts) 返回计数数组中最大值的索引，即出现次数最多的标签的索引
        return unique_labels[np.argmax(counts)]

    def score(self, X, y):
        """
        计算模型在测试集上的准确率。

        参数:
            X (numpy.ndarray): 测试特征数据，形状为 (n_samples, n_features)。
            y (numpy.ndarray): 测试标签数据，形状为 (n_samples,)。

        返回:
            float: 模型在测试集上的准确率（正确预测的样本数占总样本数的比例）。
        """
        y_pred = self.predict(X)
        accuracy = np.mean(y_pred == y)
        return accuracy

# 3、模型训练与评估

In [15]:
# 实例化KNN模型
knn = KNN(k=5)

# 训练模型
knn.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = knn.predict(X_test)

# 计算准确率
accuracy = np.mean(y_pred == y_test)
print(f"模型准确率: {accuracy:.4f}")


模型准确率: 0.7000


# 4、测试不同K值影响

In [16]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
# 尝试不同的 K 值
k_values = range(1, 21)  # 尝试 K 从 1 到 20
accuracies = []

for k in k_values:
    knn = KNN(k=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = np.mean(y_pred == y_test)
    accuracies.append(accuracy)

# 使用 Plotly 绘制交互式折线图
fig = go.Figure(data=[go.Scatter(x=list(k_values), y=accuracies, mode='lines+markers')])

fig.update_layout(
    title='KNN 模型准确率 vs. K 值',
    title_x=0.5,
    xaxis_title='K 值',
    yaxis_title='准确率',
    hovermode='closest'
)

iplot(fig) # 在 Notebook 中显示